In [1]:
import os
os.chdir("/gpfs/home/asun/jin_lab/perturbench/0_datasets")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

/gpfs/group/jin/asun/perturbench/0_datasets


In [2]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [3]:
boli = sc.read_h5ad('/gpfs/home/asun/jin_lab/perturbench/0_datasets/boli_ctx_scprocess_no+ctrl.h5ad')
boli

AnnData object with n_obs × n_vars = 21700 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident', 'Assign_clean', 'condition', 'cell_type', 'cell_class'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'

In [4]:

# start with default "train"
boli.obs["transfer_split_seed1"] = "train"

# condition = TBR1 or SATB2  AND  cell_type = L6_CT_CTX → "test"
mask_test = (
    boli.obs["condition"].isin(["TBR1", "SATB2"])
    & (boli.obs["cell_type"] == "L6_CT_CTX")
)
boli.obs.loc[mask_test, "transfer_split_seed1"] = "test"

# condition = XPO7  AND  cell_type = L6_CT_CTX → "val"
mask_val = (
    (boli.obs["condition"] == "XPO7")
    & (boli.obs["cell_type"] == "L6_CT_CTX")
)
boli.obs.loc[mask_val, "transfer_split_seed1"] = "val"

In [5]:
import numpy as np

SEED = 42
rng = np.random.default_rng(SEED)

ctrl_label = "ctrl"                 # change if your control label is different
split_col  = "transfer_split_seed1"

# Make sure the split column exists (you already set defaults above)
if split_col not in boli.obs:
    boli.obs[split_col] = "train"

# Work per cell type, only among control cells
for ct in boli.obs.loc[boli.obs["condition"] == ctrl_label, "cell_type"].unique():
    # indices (row labels) of ctrl cells for this cell type
    idx = boli.obs.index[
        (boli.obs["condition"] == ctrl_label) & (boli.obs["cell_type"] == ct)
    ].to_numpy()

    # randomize order (without replacement)
    rng.shuffle(idx)

    n = idx.size
    # split points for ~50% / 25% / 25%
    p1 = int(np.floor(0.50 * n))
    p2 = int(np.floor(0.75 * n))

    train_idx = idx[:p1]
    val_idx   = idx[p1:p2]
    test_idx  = idx[p2:]

    # assign splits
    boli.obs.loc[train_idx, split_col] = "train"
    boli.obs.loc[val_idx,   split_col] = "val"
    boli.obs.loc[test_idx,  split_col] = "test"

# (optional) make the column categorical for consistency
boli.obs[split_col] = boli.obs[split_col].astype("category")

In [ ]:
# Save CSV with transfer split information
out = boli.obs[["transfer_split_seed1"]].copy()
csv_filename = f"linear_baseline_split_ctrls.csv"
out.to_csv(csv_filename, index=True, index_label="cell_barcode", header=False)